In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import roc_auc_score as roc

from sklearn import linear_model
from sklearn import ensemble
from sklearn import svm
import xgboost as xgb
import lightgbm as lgbm

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
#ori = pd.read_csv('kindey stone urine analysis.csv')
sample = pd.read_csv('sample_submission.csv')
#train = pd.concat([train,ori],axis=0)
train.drop('id', inplace=True, axis=1)
test.drop('id', inplace=True, axis=1)

FileNotFoundError: ignored

In [ ]:
sns.heatmap(train.corr())

In [ ]:
def feature_engineering(train):    
    train["sum"] = train['calc']+train['gravity']-8.20807805874e-3*train['cond']
    if 'target' in train.columns:
        train.drop(index=[55], inplace=True)
    return train


train = feature_engineering(train)
test = feature_engineering(test) 

In [ ]:
x = """
def FE(df):
  # Ratio of calcium concentration to urea concentration: 
  df['calc_urea_ratio'] = df['calc'] / df['urea']

  # Ratio of specific gravity to osmolarity: 
  df['gravity_osm_ratio'] = df['gravity'] / df['osmo']

  # Product of calcium concentration and osmolarity: 
  df['calc_osm_product'] = df['calc'] * df['osmo']

  # Product of specific gravity and conductivity: 
  df['gravity_cond_product'] = df['gravity'] * df['cond']

  # Ratio of calcium concentration to specific gravity: 
  df['calc_gravity_ratio'] = df['calc'] / df['gravity']

  # Ratio of urea concentration to specific gravity: 
  df['urea_gravity_ratio'] = df['urea'] / df['gravity']

  # Product of osmolarity and conductivity: 
  df['osm_cond_product'] = df['osmo'] * df['cond']

  # Ratio of calcium concentration to osmolarity: 
  df['calc_osm_ratio'] = df['calc'] / df['osmo']

  # Ratio of urea concentration to osmolarity: 
  df['urea_osm_ratio'] = df['urea'] / df['osmo']

  # Product of specific gravity and urea concentration: 
  df['gravity_urea_product'] = df['gravity'] * df['urea']
FE(train)
FE(test)
"""

In [ ]:
#train.drop(index=[55], inplace=True)
train_val = train['target']
train.drop('target', axis=1, inplace=True)

In [ ]:
sc = StandardScaler()
train = sc.fit_transform(train)
test = sc.transform(test)

In [ ]:
model = SVC(kernel = 'rbf', random_state = 0)
scores = cross_val_score(model, train, train_val, cv=5, scoring='roc_auc')
scores

In [ ]:
[0.77085781, 0.79318449, 0.81316099, 0.80012077, 0.71591592]

In [ ]:
model.fit(train, train_val)

In [ ]:
pred = model.predict(test)
sample['target'] = pred

In [ ]:
model2 = LogisticRegression()
scores = cross_val_score(model2, train, train_val, cv=5, scoring='roc_auc')
scores.mean()

In [ ]:
[0.73971798, 0.73678026, 0.83431257, 0.78381643, 0.74474474]

In [ ]:
#sample.to_csv('submission.csv', index=False)

In [ ]:
model2.fit(train, train_val)

In [ ]:
pred2 = model2.predict(test)
sample['target'] = pred2

In [ ]:
print(pred.sum())
print(pred2.sum())
sample.to_csv('inioke.csv', index=False)

In [ ]:
models = {
    'logistic_regression': LogisticRegression(),
    'lasso': linear_model.Lasso(),
    'ridge': linear_model.RidgeClassifier(),
    'svc': svm.SVC(),
    'randomforest': ensemble.RandomForestClassifier(),
    'gradientboosting': ensemble.GradientBoostingClassifier(),
    'xgboost': xgb.XGBClassifier(),
    'lightgbm': lgbm.LGBMClassifier()
}

In [ ]:
def kfold_cv(model, X, y):
    kf = KFold(n_splits=5)
    return cross_val_score(model, X, y, cv=5, scoring='roc_auc').mean()

In [ ]:
best_score = -1
best = []

for name, model in models.items():
    score = kfold_cv(model, train, train_val)
    print(f'{name}: {score}')
    if score > best_score:
        best_score = score
        best = [name, score]
        
print(f'Model terbaik adalah {best[0]}: {best[1]}')

In [ ]:
best_model = models[best[0]]
best_model.fit(train, train_val)
best_pred = best_model.predict(test)
sample['target'] = best_pred
scores = cross_val_score(best_model, train, train_val, cv=5, scoring='roc_auc')
sample.to_csv('mantap.csv', index=False)
print(best_pred.sum())
sample